<a href="https://colab.research.google.com/github/euphoria96/KB_Smishing_dacon/blob/master/Model4_BiLSTM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting
Mounting google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: ...

Enter your authorization code:
··········
Mounted at /content/gdrive


### Download Mecab
Mecab for colab by dacon.io

In [3]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab

/content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

### Import packages

In [0]:
# data preprocessing
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
# tokenizer
import re
from konlpy.tag import Mecab
# modeling
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU, Bidirectional, Dropout
from tensorflow.keras.models import Sequential, model_from_json
from sklearn.metrics import roc_auc_score
import pickle
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings(action='ignore') 

### Load Data

In [6]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/

/content/gdrive/My Drive/Colab Notebooks/smishing


In [7]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/public_test.csv')
submission=pd.read_csv('data/submission_제출양식.csv')
print(train.shape, test.shape, submission.shape)
train.head(3)

(295945, 4) (1626, 3) (1626, 2)


,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0


## Data Preparation


In [0]:
def text_preprocessing(text_list):
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Mecab()
    token_list = []
    for text in text_list:
        tokens = []
        txt = re.sub('[^가-힣a-z]',' ',text.lower())
        txt = re.sub('x{1,}',' ',txt)
        token = tokenizer.morphs(txt)
        for t in token:
            if t not in stopwords:
                tokens.append(t)
        token_list.append(' '.join(tokens))
    return token_list

In [0]:
train['text_'] = text_preprocessing(train.text)
test['text_'] = text_preprocessing(test.text)

### Sampling
- Counter({0: 277242, 1: 18703}) #0.0631975535994864
- mixed sampling

In [0]:
def data_sampling(seed=1234, k=3):
    train_nsm_idx=list(train[train['smishing']==0].index)
    train_sm_idx=list(train[train['smishing']==1].index)
    random.seed(seed)
    train_nsm_idx = random.sample(train_nsm_idx, k=18703*k)
    random.seed(seed)
    train_sm_idx = random.choices(train_sm_idx, k=18703*k)
    train_idx = train_nsm_idx + train_sm_idx
    print(train_idx[:5])
    random.shuffle(train_idx)
    print(train_idx[:5])
    train_X = pd.DataFrame(train['text_'], columns=['text_']).iloc[train_idx].reset_index(drop=True)
    train_y = pd.DataFrame(train['smishing'], columns=['smishing']).iloc[train_idx].reset_index(drop=True)
    print(train_X.shape, train_y.shape)
    return train_X, train_y

### Tokenizing

In [0]:
def data_preparation(train_text, test_text, max_len=1000):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab size: ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen=max_len)
    test_X_seq = tokenizer.texts_to_sequences(test_text)
    X_test = pad_sequences(test_X_seq, maxlen=max_len)
    print("train & test shape: ", X_train.shape, X_test.shape)
    return X_train, X_test, vocab_size

In [0]:
auc_=0
def auc_score(y_true, y_pred):
  global auc_
  try:
    auc_ = roc_auc_score( y_true, y_pred, average='macro', sample_weight=None).astype('float32')
  except ValueError:
    pass
  return auc_

def auc( y_true, y_pred ) :
    score = tf.py_func( lambda y_true, y_pred : auc_score(y_true, y_pred) , [y_true, y_pred], 'float32', stateful=False, name='sklearnAUC' )
    return score

In [0]:
def model_save(model, mname):
  model_json = model.to_json()
  with open('model/'+mname+'.json', 'w') as json_file : 
      json_file.write(model_json)
  model.save_weights('model/'+mname+'.h5')

## Modeling

### 2. LSTM
##### (1) train

In [31]:
train_X, train_y = data_sampling(seed=14, k=3)

[58166, 294810, 133868, 146944, 138654]
[20507, 291834, 220048, 288135, 117900]
(112218, 1) (112218, 1)


In [35]:
X_train, X_test, vocab_size = data_preparation(train_X.text_, test.text_, max_len=800)

vocab size:  22741
train & test shape:  (112218, 800) (1626, 800)


In [0]:
def build_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = 800))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc])
    model.summary()
    return model

In [38]:
mname = 'BiLSTM_v6_ep3_'+(datetime.now() + timedelta(hours=9) ).isoformat()[:-7]

history = model.fit(X_train, train_y, epochs=3, batch_size=128, validation_split=0.2)
model_save(model, mname)

y_pred = model.predict(X_test, batch_size=128)
submission['smishing'] = y_pred
submission.to_csv('submission/'+mname+'.csv',index=False)

Train on 89774 samples, validate on 22444 samples
Epoch 1/3
89774/89774 [==============================] - 1784s 20ms/sample - loss: 0.0213 - auc: 0.9973 - val_loss: 0.0012 - val_auc: 1.0000
Epoch 2/3
89774/89774 [==============================] - 1777s 20ms/sample - loss: 2.8228e-04 - auc: 1.0000 - val_loss: 6.9778e-05 - val_auc: 1.0000
Epoch 3/3
89774/89774 [==============================] - 1735s 19ms/sample - loss: 6.0473e-05 - auc: 1.0000 - val_loss: 1.1175e-05 - val_auc: 1.0000


---

In [40]:
train_X, train_y = data_sampling(seed=1414, k=3)
X_train, X_test, vocab_size = data_preparation(train_X.text_, test.text_, max_len=800)

mname = 'BiLSTM_v6_ep3_v2_'+(datetime.now() + timedelta(hours=9) ).isoformat()[:-7]

model1 = build_model()
history = model1.fit(X_train, train_y, epochs=3, batch_size=128, validation_split=0.2)
model_save(model1, mname)

y_pred2 = model1.predict(X_test, batch_size=128)
submission['smishing'] = (y_pred + y_pred2)/2
submission.to_csv('submission/'+mname+'.csv',index=False)

[75290, 110818, 51127, 260254, 45739]
[114583, 161102, 95488, 237237, 112813]
(112218, 1) (112218, 1)
vocab size:  22783
train & test shape:  (112218, 800) (1626, 800)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 800, 128)          2916224   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 3,023,361
Trainable params: 3,023,361
Non-trainable params: 0
________

In [45]:
random.randint(0,99999999)

15026912

In [46]:
train_X, train_y = data_sampling(seed=15026912, k=3)
X_train, X_test, vocab_size = data_preparation(train_X.text_, test.text_, max_len=800)

model1 = build_model()
history = model1.fit(X_train, train_y, epochs=3, batch_size=128, validation_split=0.2)

mname = 'BiLSTM_v6_ep3_v3_'+(datetime.now() + timedelta(hours=9) ).isoformat()[:-7]
model_save(model1, mname)

y_pred = model1.predict(X_test, batch_size=128)
submission['smishing'] = y_pred
submission.to_csv('submission/'+mname+'.csv',index=False)

[206864, 218560, 111768, 152524, 170588]
[215521, 255764, 240035, 194395, 282623]
(112218, 1) (112218, 1)
vocab size:  22656
train & test shape:  (112218, 800) (1626, 800)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 800, 128)          2899968   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 3,007,105
Trainable params: 3,007,105
Non-trainable params: 0
____